In [3]:
# Import required libraries
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import nltk
import matplotlib.pyplot as plt

In [4]:
# Install Kaggle API
!pip install kaggle

# Download dataset from Kaggle
!kaggle datasets download -d jp797498e/twitter-entity-sentiment-analysis

# List the current directory to verify the downloaded zip file
!ls

# Extract dataset
from zipfile import ZipFile
dataset = 'twitter-entity-sentiment-analysis.zip'  # Update path if necessary

with ZipFile(dataset, 'r') as zip:
    zip.extractall()  # Specify a path if needed
    print('The data is extracted!')

# Download NLTK data
import nltk
nltk.download('stopwords')
nltk.download('punkt')

# Read the dataset
twitter_data = pd.read_csv('twitter_training.csv', encoding='ISO-8859-1')  # Update path if necessary


     ---------------------------------------- 0.0/82.7 kB ? eta -:--:--
     --------- ---------------------------- 20.5/82.7 kB 320.0 kB/s eta 0:00:01
     -------------------------------------- 82.7/82.7 kB 917.1 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for kaggle: filename=kaggle-1.6.17-py3-none-any.whl size=105795 sha256=34e08d207fbcbfca42ee4143008257287dcdad2a234503cd876822191ce44a4d
  Stored in directory: c:\users\louis bernal\appdata\local\pip\cache\wheels\46\d2\26\84d0a1acdb9c6baccf7d28cf06962ec80529fe1ad938489983
Successfully built kaggle
Dataset URL: https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis
License(s): CC0-1.0




  0%|          | 0.00/1.99M [00:00<?, ?B/s]
 50%|█████     | 1.00M/1.99M [00:00<00:00, 1.11MB/s]
100%|██████████| 1.99M/1.99M [00:01<00:00, 2.22MB/s]
100%|██████████| 1.99M/1.99M [00:01<00:00, 1.93MB/s]


The data is extracted!


'ls' is not recognized as an internal or external command,
operable program or batch file.
[nltk_data] Downloading package stopwords to C:\Users\Louis
[nltk_data]     Bernal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Louis
[nltk_data]     Bernal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
# Check for missing values
print(twitter_data.isnull().sum())

# Describe the dataset
print(twitter_data.describe())

# Information about the dataset
print(twitter_data.info())


2401                                                       0
Borderlands                                                0
Positive                                                   0
im getting on borderlands and i will murder you all ,    686
dtype: int64
               2401
count  74681.000000
mean    6432.640149
std     3740.423819
min        1.000000
25%     3195.000000
50%     6422.000000
75%     9601.000000
max    13200.000000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74681 entries, 0 to 74680
Data columns (total 4 columns):
 #   Column                                                 Non-Null Count  Dtype 
---  ------                                                 --------------  ----- 
 0   2401                                                   74681 non-null  int64 
 1   Borderlands                                            74681 non-null  object
 2   Positive                                               74681 non-null  object
 3   im getting on borderlands and i wil

In [6]:
# Limit dataset to 4000 rows for processing
limited_twitter_data = twitter_data.head(4000)

# Rename columns and read dataset again with specified column names
column_name = ['id', 'entity', 'sentiment', 'tweet']
twitter_data = pd.read_csv('/content/twitter_training.csv', names=column_name, encoding='ISO-8859-1')

FileNotFoundError: [Errno 2] No such file or directory: '/content/twitter_training.csv'

In [ ]:
# Check rows and columns of the dataset
print(twitter_data.shape)

# Display the first few rows of the dataset
print(twitter_data.head())

# Check the distribution of 'id' column
print(twitter_data['id'].value_counts())

In [ ]:
# Stemming function
port_stem = PorterStemmer()

def stemming(content):
    if not isinstance(content, str):
        return content

    stemmed_content = word_tokenize(content)
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if word not in stopwords.words('english')]
    return ' '.join(stemmed_content)

# Apply stemming to tweets
twitter_data['stemmed_content'] = twitter_data['tweet'].apply(stemming)
print(twitter_data.head())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Limit dataset to 2000 rows for further processing
limited_twitter_data = twitter_data.head(2000)

# Separate data and label
x = limited_twitter_data['stemmed_content'].values
y = limited_twitter_data['id'].values

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=42)

# Handle missing values in x_train and x_test
x_train = pd.Series(x_train).fillna('').astype(str)
x_test = pd.Series(x_test).fillna('').astype(str)

In [ ]:
# Vectorizer for TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
x_train_transformed = vectorizer.fit_transform(x_train)
x_test_transformed = vectorizer.transform(x_test)

# Train Logistic Regression model
model = LogisticRegression(max_iter=1000)
model.fit(x_train_transformed, y_train)

,0
2401,0
Borderlands,0
Positive,0
"im getting on borderlands and i will murder you all ,",686


In [ ]:
# Sentiment assessment function with keyword-based positive prediction
def assess_sentiment():
    positive_count = 0
    negative_count = 0
    num_sentences = 2  # Number of sentences to process
    positive_keywords = ['good', 'great', 'excellent', 'fantastic', 'amazing', 'happy', 'positive', 'love', 'joy', 'fun']

    print(f"Enter {num_sentences} sentences below:")

    for i in range(num_sentences):
        user_input = input(f"Enter sentence {i+1}: ")
        user_input_lower = user_input.lower()

        if any(keyword in user_input_lower for keyword in positive_keywords):
            print(f"{user_input} - Sentiment: Positive (Keyword-based)")
            positive_count += 1
        else:
            user_input_tfidf = vectorizer.transform([user_input])
            prediction = model.predict(user_input_tfidf)

            if prediction[0] == 1:
                print(f"{user_input} - Sentiment: Positive (Model-based)")
                positive_count += 1
            else:
                print(f"{user_input} - Sentiment: Negative")
                negative_count += 1

    # Visualize results
    visualize_sentiment(positive_count, negative_count)

    # Check model accuracy on test data
    x_test_tfidf = vectorizer.transform(x_test)
    y_pred = model.predict(x_test_tfidf)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'\nModel Accuracy: {accuracy:.2f}')

# Visualize sentiment results as a bar graph
def visualize_sentiment(positive_count, negative_count):
    labels = ['Positive', 'Negative']
    values = [positive_count, negative_count]

    plt.bar(labels, values, color=['green', 'red'])
    plt.title('Sentiment Prediction Results')
    plt.ylabel('Number of Sentences')

    for i, value in enumerate(values):
        plt.text(i, value, str(value), ha='center', va='bottom')

    plt.show()

# Run the sentiment assessment and visualization
assess_sentiment()

,2401
count,74681.000000
mean,6432.640149
std,3740.423819
min,1.000000
25%,3195.000000
50%,6422.000000
75%,9601.000000
max,13200.000000
